In [1]:
from tensorflow import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D, Input, Concatenate, Add, Activation

### CNN Blocks

We will look at the following implementations:
1. **ResNet Block:** The idea behind Residual Blocks is to provide skip connections for gradients to flow through an alternate path such that the performance of a Deep Neural Net is at least good as that of Shallower Net. It adds the input to output of block through an identity function.

2. **DenseNet Block:** Instead of adding the input as in Residual Block, DenseNet concatenates the input with output of block. Naturally with more blocks, the output size grows enormously making the layer Denser and so, it is not advised to use it on very deep networks.

3. **Bottleneck Block:** Modifying the Residual Block, it decreases the number of filters intermediately through 1 X 1 convolutions, to reduce number of parameters for subsequent 3 X 3 convolutions. Later, the number of filters are increased back to number of input filters with 1 X 1 convolutions.

4. **Inception Block:** Merging different operations in parallel by concatenating.

5. **ResNeXt Block:** Borrowing the idea of parallel processing from inception block, it runs the same operation through parallel paths and the number of parallel paths is called **cardinality**. 

In [2]:
img_size = 256

In [3]:
def num_parameters(layer, kernel_size, num_inp_channels, num_out_channels):
    if layer=='conv':
        return kernel_size * kernel_size * num_inp_channels * num_out_channels
    if layer=='pool':
        return 0

def conv_output_size(input_dim, kernel_size, stride = 1, padding = 0):
    return (input_dim + 2 * padding - kernel_size) // stride + 1

def pool_output_size(input_dim, pool_size, stride = 1, padding = 0):
    return (input_dim + 2 * padding - pool_size) // stride + 1    

### ResNet Block

In [4]:
inp = Input(shape=(img_size, img_size, 64))
x = Conv2D(64, kernel_size=(3,3), padding='same', activation='relu')(inp)
x = Conv2D(64, kernel_size=(3,3), padding='same', activation='relu')(x)
x = Add()([inp, x])
out = Activation('relu')(x)

model = Model(inputs = inp, outputs = out)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 6  0           []                               
                                4)]                                                               
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  36928       ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 64  36928       ['conv2d[0][0]']                 
                                )                                                             

2022-03-26 14:02:41.337381: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### DenseNet Block

In [5]:
inp = Input(shape=(img_size, img_size, 64))
x = Conv2D(64, kernel_size=(3,3), padding='same', activation='relu')(inp)
x = Conv2D(64, kernel_size=(3,3), padding='same', activation='relu')(x)
x = Concatenate()([inp, x])
out = Activation('relu')(x)

model = Model(inputs = inp, outputs = out)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 6  0           []                               
                                4)]                                                               
                                                                                                  
 conv2d_2 (Conv2D)              (None, 256, 256, 64  36928       ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_3 (Conv2D)              (None, 256, 256, 64  36928       ['conv2d_2[0][0]']               
                                )                                                           

### Bottleneck Block

In [6]:
inp = Input(shape=(img_size, img_size, 256))
x = Conv2D(64, kernel_size=(1,1), activation='relu')(inp)
x = Conv2D(64, kernel_size=(3,3), padding='same', activation='relu')(x)
x = Conv2D(256, kernel_size=(1,1), activation='relu')(x)
x = Add()([inp, x])
out = Activation('relu')(x)

model = Model(inputs = inp, outputs = out)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 256, 256, 2  0           []                               
                                56)]                                                              
                                                                                                  
 conv2d_4 (Conv2D)              (None, 256, 256, 64  16448       ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_5 (Conv2D)              (None, 256, 256, 64  36928       ['conv2d_4[0][0]']               
                                )                                                           

### Inception Block

In [7]:
inp = Input(shape=(img_size, img_size, 3))
x1 = Conv2D(64, kernel_size=(1,1), activation='relu')(inp)
x2 = Conv2D(64, kernel_size=(1,1), activation='relu')(inp)
x3 = MaxPool2D(pool_size=(3,3), strides=(1,1), padding='same')(inp)

x1 = Conv2D(32, kernel_size=(3,3), padding='same', activation='relu')(x1)
x2 = Conv2D(32, kernel_size=(5,5), padding='same', activation='relu')(x2)
x3 = Conv2D(32, kernel_size=(1,1), activation='relu')(x3)
x4 = Conv2D(32, kernel_size=(1,1), activation='relu')(inp)

# Depth Concatenation
out = Concatenate(axis=-1)([x1, x2, x3, x4])

model = Model(inputs = inp, outputs = out)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_7 (Conv2D)              (None, 256, 256, 64  256         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_8 (Conv2D)              (None, 256, 256, 64  256         ['input_4[0][0]']                
                                )                                                           

### ResNeXt Block

In [8]:
# cardinality - number of parallel paths
num_paths = 32

inp = Input(shape=(img_size, img_size, 256))

features = []
for _ in range(num_paths):
    x = Conv2D(4, kernel_size=(1,1), activation='relu')(inp)
    x = Conv2D(4, kernel_size=(3,3), padding='same', activation='relu')(x)
    x = Conv2D(256, kernel_size=(1,1), activation='relu')(x)
    features.append(x)
x = Add()(features)

out = Add()([x, inp])
model = Model(inputs = inp, outputs = out)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 256, 256, 2  0           []                               
                                56)]                                                              
                                                                                                  
 conv2d_13 (Conv2D)             (None, 256, 256, 4)  1028        ['input_5[0][0]']                
                                                                                                  
 conv2d_16 (Conv2D)             (None, 256, 256, 4)  1028        ['input_5[0][0]']                
                                                                                                  
 conv2d_19 (Conv2D)             (None, 256, 256, 4)  1028        ['input_5[0][0]']          